In [ ]:
# Libraries
from datetime import datetime

# Clean warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1 import AxesGrid

# Models
import gstools as gt
from pykrige.ok import OrdinaryKriging

# Outlier detection

## Load dataset

In [ ]:
timeseries = pd.read_csv('data/timeseries.csv')
timeseries.columns = [column.upper() for column in timeseries.columns]
timeseries = timeseries.sort_values(['TURBID', 'DAY', 'TMSTAMP'])

In [ ]:
location = pd.read_csv('data/location.csv')
location.columns = [column.upper() for column in location.columns]
location = location.sort_values(['TURBID'])

## Outlier detection

### Visualization

In [ ]:
def visualize_plain_timeseries_with_outliers(
    collection,
    series,
    labels,
    outliers,
    output,
    series_comparison=None,
    outliers_comparison=None,
    upper_bound=None,
    lower_bound=None
):

    # Initialize subplots
    fig, axs = plt.subplots(
        collection.shape[0], 
        1, 
        figsize=(50, 750), 
        sharex=True, 
        sharey=True
    )
    
    # Draw timeseries
    for ax, items in zip(axs, collection.to_dict(orient='index').values()):

        # Expand timeseries information
        labels_ = items[labels]

        # Outliers
        outliers_ = items[outliers]
        if outliers_comparison:
            
            outliers_comparison_ = items[outliers_comparison]
            outliers_recovered_ = ~outliers_ & outliers_comparison_

        # Timeseries
        series_ = items[series]
        indices_ = list(range(len(series_)))

        # Show temperature scatterplot
        ax.scatter(indices_, series_, s=(~outliers_) * 2)
        if series_comparison is not None and outliers_comparison is not None:
            
            series_comparison_ = items[series_comparison]
            ax.scatter(indices_, series_comparison_, s=(outliers_recovered_) * 2)

        # Show outliers
        ax.fill_between(
            indices_, 
            0, 
            1, 
            where=outliers_, 
            alpha=.3, 
            color='red',
            transform=ax.get_xaxis_transform(), 
            linewidth=0
        )

        # Fix plot
        ax.set_xlim([0, len(indices_)])
        ax.set_ylim([lower_bound, upper_bound])
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

        # Set title
        ax.set_title(f'TURBID {labels_}', loc='left')

    # Store viz
    plt.savefig(output)
    plt.show()

### External temperature

In [ ]:
# This graph is ugly, let's improve it

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 8))

# Plot temperature distribution
ax.hist(timeseries['ETMP'], 500)

# Format graph
ax.set_xlim(left=-275, right=425)
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(start, end, 25))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Set title
plt.title('External temperature values distribution')
plt.show()

In [ ]:
# Set up upper bound and lower bound for outliers
external_temperature_upper_bound = 50.0
external_temperature_lower_bound = -10.0

## Outlier labeling using bounds & z-score

In [ ]:
def external_temperature_outlier_by_value(value):
    
    # Upper bound
    if value > external_temperature_upper_bound:
        return True
    
    # Lower bound
    if value < external_temperature_lower_bound:
        return True
    
    # Missing value
    if np.isnan(value):
        return True
    
    # Nearly exactly zero
    if np.isclose(value, 0, rtol=1e-4):
        return True
    
    return False

In [ ]:
def external_temperature_outlier_by_z_score(external_temperatures):

    # Store temperature statistics
    mean = np.nanmean(external_temperatures_cube, axis=0)
    std = np.nanstd(external_temperatures_cube, axis=0)

    # Apply z-score methodology to evaluate points
    z_score = np.abs((external_temperatures_cube - mean) / std)
    return np.isnan(external_temperatures_cube) | (z_score > 2.5)

#### Outliers by values

In [ ]:
# Group timeseries by turbine to retrieve timeseries to fix
external_temperatures_by_turbine = timeseries.groupby(['TURBID'], as_index=False).agg(
    TIMESERIES=('ETMP', list)
)

In [ ]:
# Label outliers
external_temperatures_by_turbine['TIMESERIES'] = external_temperatures_by_turbine['TIMESERIES'].map(
    lambda series: np.where(
        np.array(list(map(external_temperature_outlier_by_value, series))),
        np.nan, 
        series
    )
)

#### Outliers by z-score

In [ ]:
# Store temperature cube and evaluate z-score outliers
external_temperatures_cube = np.array(external_temperatures_by_turbine['TIMESERIES'].tolist())
outliers = external_temperature_outlier_by_z_score(external_temperatures_cube)
external_temperatures_cube[outliers] = np.nan

In [ ]:
external_temperatures_by_turbine['TIMESERIES'] = list(external_temperatures_cube)
external_temperatures_by_turbine['OUTLIERS_INITIAL'] = list(np.isnan(external_temperatures_cube))

#### Visualize outliers

In [ ]:
visualize_plain_timeseries_with_outliers(
    collection=external_temperatures_by_turbine,
    series='TIMESERIES',
    labels='TURBID',
    outliers='OUTLIERS_INITIAL',
    output='viz/outliers/outliers.png',
    upper_bound=60,
    lower_bound=-15
)

## Linear outlier removal

### External temperature

In [ ]:
def fill_outlier_linear(timeseries):
    
    # Backup timeseries to avoid overlapping of reference
    series = timeseries.copy()
    
    # Define one step window
    for i in range(len(series) - 3):
        
        # Extract current indices to be 
        indices = list(range(i, i + 4))
        window = series[indices]
        
        # Evaluate outliers
        outliers_pattern = tuple(np.isnan(window))
        
        # [V, X, X, V]
        if outliers_pattern == (False, True, True, False):
            
            # Fill temperatures with two-steps gradient
            gradient = (window[-1] - window[0]) / 3
            window[1] = window[0] + gradient
            window[2] = window[1] + gradient
            
        # [V, X, V, V]
        elif outliers_pattern == (False, True, False, False):
            
            # Fill temperatures with two-steps gradient
            gradient = (window[2] - window[0]) / 2
            window[1] = window[0] + gradient
          
        # [V, V, X, V]
        elif outliers_pattern == (False, False, True, False):
            
            # Fill temperatures with two-steps gradient
            gradient = (window[-1] - window[1]) / 2
            window[1] = window[1] + gradient
    
        # Update timeseries values
        series[indices] = window
        
    # Return linearly filled timeseries
    return series

In [ ]:
external_temperatures_by_turbine['TIMESERIES_LINEAR_FILL'] = external_temperatures_by_turbine['TIMESERIES'].map(fill_outlier_linear)
external_temperatures_by_turbine['OUTLIERS_LINEAR'] = external_temperatures_by_turbine['TIMESERIES_LINEAR_FILL'].map(np.isnan)

In [ ]:
visualize_plain_timeseries_with_outliers(
    collection=external_temperatures_by_turbine,
    series='TIMESERIES',
    series_comparison='TIMESERIES_LINEAR_FILL',
    labels='TURBID',
    outliers='OUTLIERS_LINEAR',
    outliers_comparison='OUTLIERS_INITIAL',
    output='viz/outliers/outliers_linear.png',
    upper_bound=60,
    lower_bound=-15
)

## Kriging

### Build grid

In [ ]:
# Define grid to index assignment to draw heatmap 
grid_assignment = {
    (upper_bound, left_bound, lower_bound, right_bound): (y, x)
    for y, (lower_bound, upper_bound) in enumerate(zip(reversed(np.arange(0, 12400, 400)), reversed(np.arange(400, 12800, 400))))
    for x, (left_bound, right_bound) in enumerate(zip(np.arange(0, 5600, 400), np.arange(400, 6000, 400)))
}

# Define turbine to index assignment to draw heatmap 
turbine_assignment = {
    int(row['TURBID']): [
        grid_assignment[upper_bound, left_bound, lower_bound, right_bound]
        for upper_bound, left_bound, lower_bound, right_bound in grid_assignment.keys()
        if upper_bound >= row['Y'] and lower_bound <= row['Y'] and right_bound >= row['X'] and left_bound <= row['X']
    ].pop()
    for _, row in location.iterrows()
}

turbine_assignment_reversed = {
    value: key
    for key, value in turbine_assignment.items()
}

### Color map

In [ ]:
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero.

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower offset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax / (vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highest point in the colormap's range.
          Defaults to 1.0 (no upper offset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap

In [ ]:
vlag = sns.color_palette('vlag', as_cmap=True)
temperature_cmap = shiftedColorMap(vlag, start=0, midpoint=.18, stop=1, name='vlag_custom')
energy_cmap = sns.color_palette('rocket', as_cmap=True)

### Labels

In [ ]:
# Define heatmap labels
heatmap_labels = np.zeros((31, 14)).astype(str)

for y, row in enumerate(heatmap_labels):
    for x, col in enumerate(row):
        heatmap_labels[y, x] = str(turbine_assignment_reversed.get((y, x), ''))

### Kriging interpolation

In [ ]:
def variogram_inference(x, y, z):
    
    # Bin data and perform variogram estimation
    bins = gt.standard_bins((x, y))
    bin_c, vario = gt.vario_estimate((x, y), z, bins)
    
    # Fit variogram
    model = gt.Gaussian(dim=2)
    model.fit_variogram(bin_c, vario, nugget=False)
    
    # Return model
    return model

In [ ]:
def kriging_interpolation(x, y, z, x_grid, y_grid):
    
    # Store interpolation pairs
    z = z.copy()
    interpolation_pairs = []
    
    # Iterative kriging based on previous timeseries point
    for series_index in range(z.shape[1] - 1):
    
        # Convert series to array
        z_left = z[:, series_index]
        z_right = z[:, series_index + 1]

        # Store number of missing values
        missing_values_left = np.isnan(z_left)
        missing_values_right = np.isnan(z_right)

        # If missing values are too many, just drop this iteration
        if sum(missing_values_right) / len(z_right) > 0.5:
            continue

        # Use left kriging to improve temperature interpolation
        if not np.any(missing_values_left):

            # Variogram inference
            variogram_left = variogram_inference(x, y, z_left)
            
            # Initiliaze left Kriging
            kriging_left = OrdinaryKriging(
                x=x[~missing_values_left], 
                y=y[~missing_values_left], 
                z=z_left[~missing_values_left], 
                coordinates_type='euclidean', 
                variogram_model=variogram_left
            )

            # Perform left Kriging
            z_left_interpolated, _ = kriging_left.execute("grid", x_grid, y_grid)
            z_left_interpolated = np.flip(z_left_interpolated, axis=0)
            
        # Variogram inference
        variogram_right = variogram_inference(x, y, z_right)
            
        # Initiliaze right Kriging
        kriging_right = OrdinaryKriging(
            x=x[~missing_values_right], 
            y=y[~missing_values_right], 
            z=z_right[~missing_values_right], 
            coordinates_type='euclidean', 
            variogram_model=variogram_right
        )

        # Perform right Kriging
        z_right_interpolated, _ = kriging_right.execute("grid", x_grid, y_grid)
        z_right_interpolated = np.flip(z_right_interpolated, axis=0)

        # Sample from grid 
        z_right_interpolated_collapsed = np.array([
            z_right_interpolated[indices[0], indices[1]] 
            for indices in turbine_assignment_reversed
        ])
        
        interpolation = z_right_interpolated_collapsed.copy()

        # Perform scaling on left Kriging if available
        if not np.any(missing_values_left):

            z_left_interpolated_collapsed = np.array([
                z_left_interpolated[indices[0], indices[1]] 
                for indices in turbine_assignment_reversed
            ])
            
            # Use high autocorrelation at time-step t - 1 to perform better interpolation
            interpolation = z_left + (z_right_interpolated_collapsed - z_left_interpolated_collapsed)

        # Store interpolation 
        z_filled = np.where(missing_values_right, interpolation, z_right)
        z[:, series_index + 1] = z_filled
        
        # Store interpolation pairs
        interpolation_pairs.append((
            z_left_interpolated_collapsed if not np.any(missing_values_left) else None,
            z_right_interpolated_collapsed
        ))

    return z, interpolation_pairs

In [ ]:
# Store windmills coordinates
x = location['X'].values / 1000
y = location['Y'].values / 1000
z = np.array(external_temperatures_by_turbine['TIMESERIES_LINEAR_FILL'].tolist())

# Store windmills grid
x_grid = np.arange(0, 5.6, .4).astype(float)
y_grid = np.arange(0, 12.4, .4).astype(float)

In [ ]:
# Perform Kriging interpolation
z_iterpolated, interpolation_pairs = kriging_interpolation(x, y, z, x_grid, y_grid)

In [ ]:
external_temperatures_by_turbine['TIMESERIES_KRIGING_FILL'] = list(z_iterpolated)
external_temperatures_by_turbine['OUTLIERS_KRIGING'] = list(np.isnan(z_iterpolated))

In [ ]:
visualize_plain_timeseries_with_outliers(
    collection=external_temperatures_by_turbine,
    series='TIMESERIES',
    series_comparison='TIMESERIES_KRIGING_FILL',
    labels='TURBID',
    outliers='OUTLIERS_KRIGING',
    outliers_comparison='OUTLIERS_INITIAL',
    output='viz/outliers/outliers_kriging.png',
    upper_bound=60,
    lower_bound=-15
)

### Kriging interpolation on normalized dataset

In [ ]:
# Store windmills coordinates
x = location['X'].values.reshape(-1, 1) / 1000
y = location['Y'].values.reshape(-1, 1) / 1000
z = np.array(external_temperatures_by_turbine['TIMESERIES_LINEAR_FILL'].tolist())

# Store windmills grid
x_grid = np.arange(0, 5.6, .4).astype(float)
y_grid = np.arange(0, 12.4, .4).astype(float)

In [ ]:
# Store temperatures cube and initialize filled collection
z_normalized = z.copy()

# Extract normalization values
mean = np.nanmean(z_normalized, axis=1).reshape(-1, 1)
std = np.nanstd(z_normalized, axis=1).reshape(-1, 1)
std[np.isnan(std) | (std == 0) | (std < 12)] = np.nanmean(std[std >= 12])

# Perform normalization
z_normalized = (z_normalized - mean) / std

In [ ]:
# Perform Krigin interpolation
z_iterpolated, interpolation_pairs = kriging_interpolation(x, y, z_normalized, x_grid, y_grid)

In [ ]:
# Recover statistics
mean_r = mean.copy()
std_r = std.copy()
mean_r[np.isnan(mean_r)] = np.nanmean(mean_r)

In [ ]:
external_temperatures_by_turbine['TIMESERIES_KRIGING_FILL_NORMALIZED'] = list(z_iterpolated * std_r + mean_r)

In [ ]:
visualize_plain_timeseries_with_outliers(
    collection=external_temperatures_by_turbine,
    series='TIMESERIES',
    series_comparison='TIMESERIES_KRIGING_FILL_NORMALIZED',
    labels='TURBID',
    outliers='OUTLIERS_KRIGING',
    outliers_comparison='OUTLIERS_INITIAL',
    output='viz/outliers/outliers_kriging_normalized.png',
    upper_bound=60,
    lower_bound=-15
)